In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import os
from scipy.spatial import distance
import pickle

RootPath = r'D:\Great_job_of_teammate'
user_action_path = os.path.join(RootPath, "wechat_algo_data1/user_action.csv")
feed_info_modified_path = os.path.join(RootPath, "feed_info_modified2.pkl")
test_data_path = os.path.join(RootPath, "wechat_algo_data1/test_a.csv")
user_interest_path = os.path.join(RootPath, "user_interest.pkl")
LABEL_COLUMNS = ['click_avatar', 'forward', 'follow', 'favorite', 'read_comment', 'comment', 'like']
ACTION_LIST = ['click_avatar', 'forward', 'follow', 'favorite', 'read_comment', 'comment', 'like', 'is_stay',
               'is_finished']
with open(feed_info_modified_path, 'rb') as file:
    feed_info_modified_df = pickle.load(file)

with open(user_interest_path, 'rb') as file:
    user_interest_df = pickle.load(file)
# 读取用户历史行为数据
user_action_df = pd.read_csv(user_action_path)
# 读取测试集数据
test_data_df = pd.read_csv(test_data_path)
# 统计视频第一次出现的日期，作为视频的发布日期
feed_release_date = user_action_df.groupby(['feedid'])['date_'].apply(lambda x: x.min())
feed_release_date = pd.DataFrame(feed_release_date)
feed_info_modified_df = pd.merge(feed_info_modified_df, feed_release_date, how='left', on='feedid')
# 将没有在之前出现过的视频发布日期设置为15
feed_info_modified_df['date_'].fillna(15, inplace=True)
feed_info_modified_df.rename(columns={'date_': 'release_date_'}, inplace=True)
feed_info_df = feed_info_modified_df.copy()
feed_info_df.set_index(['feedid'], inplace=True)
# 统一play和stay的单位：
user_action_df['play'] = user_action_df['play'] / 1000.0
user_action_df['stay'] = user_action_df['stay'] / 1000.0
# 标记用户是否产生过反馈：

user_action_df['feedback'] = (user_action_df[LABEL_COLUMNS].sum(axis=1) > 0).astype(np.int)
unique_user_df = pd.DataFrame(user_action_df['userid'].drop_duplicates())

In [2]:
path=os.path.join(RootPath,'feed_info_modified1.csv')
feed_info_df=pd.read_csv(path)
print(type(feed_info_df.iloc[0]['feed_embedding'][0]))
from ast import literal_eval
tmp=feed_info_df[['feedid']]
tmp['feed_embedding'] = feed_info_df['feed_embedding'].apply(lambda x: literal_eval(x))
print(type(tmp.loc[0,'feed_embedding'][0]))

<class 'str'>
<class 'float'>


D:\anacanda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [3]:
feed_info_df=tmp
feed_info_df['feed_embedding'] = feed_info_df['feed_embedding'].apply(lambda x: np.array(x, np.float32))

In [4]:
print(user_action_df.columns)
print(feed_info_modified_df.columns)

Index(['userid', 'feedid', 'date_', 'device', 'read_comment', 'comment',
       'like', 'play', 'stay', 'click_avatar', 'forward', 'follow', 'favorite',
       'feedback'],
      dtype='object')
Index(['feedid', 'authorid', 'videoplayseconds', 'bgm_song_id',
       'bgm_singer_id', 'author_n_feeds', 'videolength_bucket',
       'feed_embedding', 'tag_list', 'keyword_list', 'embedding_0',
       'embedding_1', 'embedding_2', 'embedding_3', 'embedding_4',
       'embedding_5', 'embedding_6', 'embedding_7', 'embedding_8',
       'embedding_9', 'embedding_10', 'embedding_11', 'embedding_12',
       'embedding_13', 'embedding_14', 'embedding_15', 'embedding_16',
       'embedding_17', 'embedding_18', 'embedding_19', 'embedding_20',
       'embedding_21', 'embedding_22', 'embedding_23', 'embedding_24',
       'embedding_25', 'embedding_26', 'embedding_27', 'embedding_28',
       'embedding_29', 'embedding_30', 'embedding_31', 'author_n_feeds_bucket',
       'release_date_'],
      dtype='obj

In [5]:
feed_info_modified_df=feed_info_modified_df[['feedid', 'authorid', 'videoplayseconds']]
print(feed_info_modified_df.columns)


Index(['feedid', 'authorid', 'videoplayseconds'], dtype='object')


In [6]:
user_action_df=pd.merge(user_action_df,feed_info_modified_df,on=['feedid'],how='left')
user_action_df['is_finished'] = (user_action_df['play'] >= user_action_df['videoplayseconds']).astype(np.int)
user_action_df['is_stay'] = (user_action_df['stay'] >= user_action_df['videoplayseconds'] * 2.0).astype(
        np.int)

In [7]:
#别忘了设置index！
feed_info_df.set_index(['feedid'], inplace=True)

In [12]:
def generate_user_embedding(user_action_df):

    def feed_embedding_mean(feedid):
        if len(feedid)>0:
            a=feed_info_df.loc[feedid]['feed_embedding'].mean()
        else:
            print("!")
            a=np.zeros((512,))
        return a 
        

    user_embedding_df=user_action_df[user_action_df[ACTION_LIST].sum(axis=1)>0].groupby('userid')['feedid'].unique().reset_index()
    #print(user_embedding_df.head())
    user_embedding_df['user_embedding'] = user_embedding_df['feedid'].apply(lambda x: feed_embedding_mean(x))
    user_id=user_action_df[['userid']].drop_duplicates(['userid'], keep='first', inplace=False)
    #print(user_id.shape)
    user_embedding_df=pd.merge(user_id,user_embedding_df,on=['userid'],how='left')
    #print(user_embedding_df.columns)
    num_of_na=user_embedding_df['user_embedding'].isnull().sum()
    fill_null=np.zeros((512,))
    for i in user_embedding_df[user_embedding_df['user_embedding'].isnull().values==True].index.tolist():
        user_embedding_df.set_value(i, 'user_embedding',fill_null)
    user_embedding_df.drop(columns=['feedid'],inplace=True)
    print(user_embedding_df['user_embedding'].isnull().sum())
    return user_embedding_df

In [13]:
user_action_train_split = user_action_df[(user_action_df['date_'] >= 1) & (user_action_df['date_'] <= 7)]
#user_action_val_split = user_action_df[user_action_df['date_'] == date + days + 1][['userid', 'feedid', 'device'] + LABEL_COLUMNS]
user_embedding=generate_user_embedding(user_action_train_split)      

Index(['userid', 'feedid', 'user_embedding'], dtype='object')
0


D:\anacanda\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [14]:
print(user_embedding['user_embedding'].isnull().sum())

0


In [15]:
print(feed_info_modified_df.columns)

Index(['feedid', 'authorid', 'videoplayseconds'], dtype='object')


In [16]:
def generate_author_embedding():
    def feed_embedding_mean(feedid):
        try:
            a=feed_info_df.loc[feedid]['feed_embedding'].mean()
            return a 
        except KeyError:
            print(feedid)
    author_embedding_df=feed_info_modified_df.groupby('authorid')['feedid'].unique().reset_index()
    author_embedding_df['author_embedding']=author_embedding_df['feedid'].apply(lambda x: feed_embedding_mean(x))
    author_id=feed_info_modified_df[['authorid']].drop_duplicates(['authorid'], keep='first', inplace=False)
    author_embedding_df=pd.merge(author_id,author_embedding_df,on=['authorid'],how='left')
    print(author_embedding_df.columns)
    author_embedding_df.drop(columns=['feedid'],inplace=True)
    save_path=os.path.join(RootPath,'author_embedding.pkl')
    #print(author_embedding_df.head())
    with open(save_path,'wb') as f:
        pickle.dump(author_embedding_df,f)
    return author_embedding_df
    
result=generate_author_embedding()

Index(['authorid', 'feedid', 'author_embedding'], dtype='object')


In [ ]:
save_path=os.path.join(RootPath,'author_embedding.pkl')
with open(save_path,'rb') as f:
    result=pickle.load(f)
print(type((result.iloc[0]['author_embedding'])))

In [ ]:
print(type((user_embedding.iloc[0]['user_embedding'])))
print(user_embedding.columns)

In [ ]:
def Split_and_Process(user_action_df, days): 
  """
  days: 滑动窗口的大小
  """
  for date in range(14 - days):
    user_action_train_split = user_action_df[(user_action_df['date_'] >= 1) & (user_action_df['date_'] <= date + days)]
    user_embedding=generate_user_embedding(user_action_train_split)
    save_path=os.path.join(RootPath,'user_embedding_{}.pkl'.format(date+days+1))
    with open(save_path,'wb') as f:
        pickle.dump(user_embedding,f)
    print()

In [ ]:
Split_and_Process(user_action_df,7)